In [3]:
import tensorflow as tf


samples_file_path = "modelSamples.csv"

In [4]:
def get_dataset(file_path):
    dataset = tf.data.experimental.make_csv_dataset(
        file_path,
        batch_size=12,
        label_name='label',
        na_value="?",
        num_epochs=1,
        ignore_errors=True)
    return dataset



# sample dataset size 110830/12(batch_size) = 9235
raw_samples_data = get_dataset(samples_file_path)


test_dataset = raw_samples_data.take(1000)
train_dataset = raw_samples_data.skip(1000)

Instructions for updating:
Use `tf.data.Dataset.ignore_errors` instead.


In [7]:
genre_vocab = ['Film-Noir', 'Action', 'Adventure', 'Horror', 'Romance', 'War', 'Comedy', 'Western', 'Documentary',
               'Sci-Fi', 'Drama', 'Thriller',
               'Crime', 'Fantasy', 'Animation', 'IMAX', 'Mystery', 'Children', 'Musical']


GENRE_FEATURES = {
    'userGenre1': genre_vocab,
    'userGenre2': genre_vocab,
    'userGenre3': genre_vocab,
    'userGenre4': genre_vocab,
    'userGenre5': genre_vocab,
    'movieGenre1': genre_vocab,
    'movieGenre2': genre_vocab,
    'movieGenre3': genre_vocab
}


categorical_columns = []
for feature, vocab in GENRE_FEATURES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
    emb_col = tf.feature_column.embedding_column(cat_col, 10)
    categorical_columns.append(emb_col)


movie_col = tf.feature_column.categorical_column_with_identity(key='movieId', num_buckets=1001)
movie_emb_col = tf.feature_column.embedding_column(movie_col, 10)
categorical_columns.append(movie_emb_col)


user_col = tf.feature_column.categorical_column_with_identity(key='userId', num_buckets=30001)
user_emb_col = tf.feature_column.embedding_column(user_col, 10)
categorical_columns.append(user_emb_col)

In [8]:
numerical_columns = [tf.feature_column.numeric_column('releaseYear'),
                   tf.feature_column.numeric_column('movieRatingCount'),
                     tf.feature_column.numeric_column('movieAvgRating'),
                     tf.feature_column.numeric_column('movieRatingStddev'),
                     tf.feature_column.numeric_column('userRatingCount'),
                     tf.feature_column.numeric_column('userAvgRating'),
                     tf.feature_column.numeric_column('userRatingStddev')]


Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


In [9]:
preprocessing_layer = tf.keras.layers.DenseFeatures(numerical_columns + categorical_columns)


model = tf.keras.Sequential([
    preprocessing_layer,
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

In [10]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

In [12]:
model.fit(train_dataset, epochs=10)
test_loss, test_accuracy = model.evaluate(test_dataset)
print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

Epoch 1/10
8232/8232 [==============================] - 153s 18ms/step - loss: 2.7467 - accuracy: 0.5805
Epoch 2/10
8232/8232 [==============================] - 145s 18ms/step - loss: 0.6289 - accuracy: 0.6711
Epoch 3/10
8232/8232 [==============================] - 145s 17ms/step - loss: 0.5562 - accuracy: 0.7185
Epoch 4/10
8232/8232 [==============================] - 144s 17ms/step - loss: 0.5230 - accuracy: 0.7404
Epoch 5/10
8232/8232 [==============================] - 149s 18ms/step - loss: 0.5054 - accuracy: 0.7524
Epoch 6/10
8232/8232 [==============================] - 147s 18ms/step - loss: 0.4920 - accuracy: 0.7606
Epoch 7/10
8232/8232 [==============================] - 141s 17ms/step - loss: 0.4839 - accuracy: 0.7662
Epoch 8/10
8232/8232 [==============================] - 143s 17ms/step - loss: 0.4770 - accuracy: 0.7689
Epoch 9/10
8232/8232 [==============================] - 142s 17ms/step - loss: 0.4713 - accuracy: 0.7711
Epoch 10/10
1000/1000 [==============================] 